In [5]:
import numpy as np
import torch
from torch.autograd import Variable

In [14]:
w1=Variable(torch.randn(10,4,dtype=torch.float64),requires_grad=True)
w2=Variable(torch.randn(4,4,dtype=torch.float64),requires_grad=True)
learning_rate=1e-5

In [15]:
def CRF(num_labels,n,input_dataset,labels):
    input_dataset=Variable(input_dataset,requires_grad=False)
    Z=input_dataset[0].mm(w1)
    sum_exp=Z[labels[0]]
    for i in range(n-1):
        x=torch.zeros(1,num_labels)
        x[0,labels[i+1]]=1
        score1=x.mm(w2)[labels[i+1]]
        score2=input_dataset[i+1].mm(w1)[labels[i+1]]
        sum_exp=sum_exp+score1+score2
    up_score=torch.exp(sum_exp)
    
    G=torch.exp(w2)
    
    for i in range(n-1):
        Z+=(Z.mm(G)).mul(torch.exp(input_dataset[i+1].mm(w1)))
    all_Z=torch.sum(Z)
    
    p=up_score/all_Z
    
    loss=-torch.log(p)
    
    loss.backward()
    
    w1.data-=learning_rate*w1.grad.data
    w2.data-=learning_rate*w2.grad.data
        
    w1.grad.data.zero_()
    w2.grad.data.zero_()
    
    
    
    
    

In [16]:
def train(epochs,all_data):
    for i in range(epochs):
        
        for input_dataset,labels in all_data:
            CRF(num_labels,n,input_dataset,labels)
    return w1,w2


In [ ]:
w1,w2=train(epochs,all_data)

In [19]:
def find_best(input_dataset,n,true_labels):
    input_dataset=Variable(input_dataset,requires_grad=False)
    list_Z=[]
    
    G=torch.exp(w2)
    Z=input_dataset[0].mm(w1)
    list_Z.append(Z)
    for i in range(n-1):
        Z=(Z.mm(G)).mul(torch.exp(input_dataset[i+1].mm(w1)))
        list_Z.append(Z)
    
    
    list_label=[]
    
    for i in range(len(list_Z)):
        list_label.append(true_labels[list_Z.argmax().numpy()[0]])
    return list_label
        
    